Mean across cross-val runs

In [1]:
from os.path import join
from os import listdir
import torch
import pandas as pd

In [2]:
configs = [
    'competitions/2020/melanoma-classification/configs/effb5/best-1cycle-wd4e-1-384',
    'competitions/2020/melanoma-classification/configs/effb5/best-1cycle-wd4e-1-384/fold2',
    'competitions/2020/melanoma-classification/configs/effb5/best-1cycle-wd4e-1-384/fold3',
    'competitions/2020/melanoma-classification/configs/effb5/best-1cycle-wd4e-1-384/fold4',
    'competitions/2020/melanoma-classification/configs/effb5/best-1cycle-wd4e-1-384/fold5']

In [4]:
all_predictions = None

for index, config in enumerate(configs):
    pred_path = join('/output', config, 'logs/evaluation/test.csv')
    fold_predictions = pd.read_csv(pred_path)
    fold_predictions = fold_predictions.rename(columns={'target': f'target{index+1}'})
    if all_predictions is None:
        all_predictions = fold_predictions
    else:
        all_predictions = pd.merge(all_predictions, fold_predictions, on='image_name')

In [5]:
all_predictions

,image_name,target1,target2,target3,target4,target5
0,ISIC_0052060,0.004660,0.002089,0.000561,0.002501,0.000972
1,ISIC_0052349,0.001814,0.001551,0.002794,0.001903,0.000376
2,ISIC_0058510,0.000735,0.002648,0.001007,0.002056,0.001774
3,ISIC_0073313,0.000557,0.001270,0.006088,0.003615,0.000416
4,ISIC_0073502,0.001607,0.001718,0.004602,0.001748,0.003551
...,...,...,...,...,...,...
10977,ISIC_9992485,0.013135,0.001588,0.006692,0.006469,0.002189
10978,ISIC_9996992,0.350060,0.577967,0.400357,0.281233,0.550544
10979,ISIC_9997917,0.390329,0.181306,0.157224,0.050817,0.020641
10980,ISIC_9998234,0.006581,0.011548,0.001433,0.005583,0.004646


In [6]:
target_columns = [f'target{index+1}' for index in range(len(configs))]

In [7]:
target_columns

['target1', 'target2', 'target3', 'target4', 'target5']

In [8]:
all_predictions['target'] = all_predictions[target_columns].mean(axis=1)

In [9]:
all_predictions

,image_name,target1,target2,target3,target4,target5,target
0,ISIC_0052060,0.004660,0.002089,0.000561,0.002501,0.000972,0.002157
1,ISIC_0052349,0.001814,0.001551,0.002794,0.001903,0.000376,0.001688
2,ISIC_0058510,0.000735,0.002648,0.001007,0.002056,0.001774,0.001644
3,ISIC_0073313,0.000557,0.001270,0.006088,0.003615,0.000416,0.002389
4,ISIC_0073502,0.001607,0.001718,0.004602,0.001748,0.003551,0.002645
...,...,...,...,...,...,...,...
10977,ISIC_9992485,0.013135,0.001588,0.006692,0.006469,0.002189,0.006015
10978,ISIC_9996992,0.350060,0.577967,0.400357,0.281233,0.550544,0.432032
10979,ISIC_9997917,0.390329,0.181306,0.157224,0.050817,0.020641,0.160063
10980,ISIC_9998234,0.006581,0.011548,0.001433,0.005583,0.004646,0.005958


In [10]:
all_predictions = all_predictions.drop(columns=target_columns)

In [11]:
all_predictions

,image_name,target
0,ISIC_0052060,0.002157
1,ISIC_0052349,0.001688
2,ISIC_0058510,0.001644
3,ISIC_0073313,0.002389
4,ISIC_0073502,0.002645
...,...,...
10977,ISIC_9992485,0.006015
10978,ISIC_9996992,0.432032
10979,ISIC_9997917,0.160063
10980,ISIC_9998234,0.005958


In [12]:
all_predictions.to_csv('submissions/effnet-b5-ns-balanced-affine-1cycle-wd-scheduler-cv-mean.csv', index=False)